In [1]:
import random

random.seed(42)

# SPDZ

In [2]:
#
# Use e.g. https://www.compilejava.net/
#
#import java.util.*;
#import java.math.*;
#
#public class Entrypoint
#{
#  public static void main(String[] args)
#  {
#    BigInteger q = BigInteger.probablePrime(128, new Random());    
#    BigInteger inverse = new BigInteger("10000000000").modInverse(q);
#    System.out.println(q);
#    System.out.println(inverse);
#  }
#}

BASE = 10

#PRECISION_INTEGRAL = 1
#PRECISION_FRACTIONAL = 6
#Q = 10000019

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert Q > BASE**PRECISION

## Sharing and reconstruction

In [3]:
def share(secret):
    first  = random.randrange(Q)
    second = (secret - first) % Q
    return [first, second]

def reconstruct(shares):
    return sum(shares) % Q

## Linear operations

In [4]:
def add(x, y):
    return [ (xi + yi) % Q for xi, yi in zip(x, y) ]

def sub(x, y):
    return [ (xi - yi) % Q for xi, yi in zip(x, y) ]

def scalar_add(x, k):
    return add(x, [k, 0])

def scalar_mul(x, k):
    return [ (xi * k) % Q for xi in x ]

a = share(5)
b = share(7)
c = add(a, b)
assert reconstruct(c) == 12

## Multiplication

In [5]:
def generate_multiplication_triple():
    a = random.randrange(Q)
    b = random.randrange(Q)
    c = a * b % Q
    return share(a), share(b), share(c)

(a, b, c) = generate_multiplication_triple()
assert reconstruct(a) * reconstruct(b) % Q == reconstruct(c)

In [6]:
def mul(x, y, triple):
    a, b, c = triple
    # local masking
    v = sub(x, a)
    w = sub(y, b)
    # communication: the players simultanously send their shares to the other
    epsilon = reconstruct(v)
    delta = reconstruct(w)
    # local combination
    r = epsilon * delta % Q
    s = scalar_mul(b, epsilon)
    t = scalar_mul(a, delta)
    return add(s, add(t, scalar_add(c, r)))

x = share(2)
y = share(5)
z = mul(x, y, generate_multiplication_triple())
assert reconstruct(z) == 10

## Exponentiation

In [7]:
def generate_exponentiation_triple(exponent):
    a = random.randrange(Q)
    return [ share(pow(a, e, Q)) for e in range(1, exponent+1) ]

a, aa = generate_exponentiation_triple(2)
assert(reconstruct(a) * reconstruct(a) % Q == reconstruct(aa))

a, aa, aaa = generate_exponentiation_triple(3)
assert(reconstruct(a) * reconstruct(a) * reconstruct(a) % Q == reconstruct(aaa))

In [8]:
from scipy.misc import comb as binom
from functools import reduce

ONE = [1,0] # constant sharing of 1

def exp(x, exponent, triple):
    # local masking
    v = sub(x, triple[0])
    # communication: the players simultanously send their share to the other
    epsilon = reconstruct(v)
    # local combination
    cs = [ binom(exponent, k, exact=True) for k in range(exponent+1) ]
    es = [ pow(epsilon, e, Q) for e in reversed(range(exponent+1)) ]
    bs = [ONE] + triple
    return reduce(add, ( scalar_mul(b, c * e) for c,e,b in zip(cs, es, bs) ))

x = share(4)
y = exp(x, 2, generate_exponentiation_triple(2))
z = exp(x, 10, generate_exponentiation_triple(10))
assert reconstruct(y) == pow(4, 2, Q)
assert reconstruct(z) == pow(4, 10, Q)

## Fixed-point encoding

In [9]:
def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

In [10]:
# using trick from SecureML paper that only requires a local operation

def truncate(x):
    y0 = x[0] // BASE**PRECISION_FRACTIONAL
    y1 = Q - ((Q - x[1]) // BASE**PRECISION_FRACTIONAL)
    return [y0, y1]

x = share(encode(.5))

a = truncate(mul(x, x, generate_multiplication_triple()))
b = truncate(mul(a, a, generate_multiplication_triple()))
v = truncate(mul(b, b, generate_multiplication_triple()))
assert decode(reconstruct(v)) == (.5)**8

d = mul(x, x, generate_multiplication_triple())
e = mul(d, d, generate_multiplication_triple())
w = mul(e, e, generate_multiplication_triple())
print(decode(reconstruct(w)))
#assert decode(reconstruct(w)) == (.5)**8

1.34658068183981e+30


## Wrap-up

In [11]:
def multiplication_triple_generator():
    while True:
        yield generate_multiplication_triple()

In [12]:
multiplication_triple_source = multiplication_triple_generator()

class SecureRational(object):
    
    def secure(secret):
        z = SecureRational()
        z.shares = share(encode(secret))
        return z
    
    def reveal(self):
        return decode(reconstruct(self.shares))
    
    def __repr__(self):
        return "SecureRational(%f)" % self.reveal()
    
    def __add__(x, y):
        z = SecureRational()
        z.shares = add(x.shares, y.shares)
        return z
    
    def __sub__(x, y):
        z = SecureRational()
        z.shares = sub(x.shares, y.shares)
        return z
    
    def __mul__(x, y):
        z = SecureRational()
        triple = multiplication_triple_source.__next__()
        z.shares = truncate(mul(x.shares, y.shares, triple))
        return z
    
    def __pow__(x, e):
        # TODO
        z = SecureRational.secure(1)
        for _ in range(e):
            z = z * x
        return z

In [13]:
x = SecureRational.secure(.5)
y = SecureRational.secure(-.25)

z = x * y
assert z.reveal() == (.5) * (-.25)

z = x**8
assert z.reveal() == (.5)**8

# Dump

## Security of mul

In [14]:
a, b, c = generate_multiplication_triple()

x = share(3)
y = share(4)

In [15]:
e = sub(x, a)
d = sub(y, b)

epsilon = reconstruct(e)
delta = reconstruct(d)

r = epsilon * delta % Q
s = scalar_mul(b, epsilon)
t = scalar_mul(a, delta)

z = add(s, add(t, scalar_add(c, r)))

In [16]:
z0_real = (c[0] + b[0] * epsilon + a[0] * delta + epsilon * delta) % Q
z0_sim = (12 - z[1]) % Q

assert z0_real == z0_sim

## Security of add

In [17]:
x = share(2)
y = share(5)
z = add(x, y)

In [18]:
z0_real = (x[0] + y[0]) % Q
z0_sim = (7 - z[1]) % Q

assert z0_real == z0_sim